In [1]:
import pandas as pd
import requests

C:\Users\sahar\anconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
with open('dests.txt','r') as file:
    countries= [line.strip() for line in file.readlines()]

In [3]:
origin = 'Tel Aviv Yafo'
api_key = 'Insert Your API"

In [4]:
def find_d(destination):
    url="https://maps.googleapis.com/maps/api/distancematrix/json?origins=%s&destinations=%s&key=%s" % (destination, origin, api_key)
    try:
        response = requests.get(url)
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        else:
            try:
                response_data = response.json()
            except:
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")
    distance = response_data['rows'][0]['elements'][0]['distance']['value']//1000
    minutes = response_data['rows'][0]['elements'][0]['duration']['value']//60
    duration = "%s hours and %s minutes" % (minutes//60, minutes % 60)
    return [distance, duration]
    

In [5]:
df = pd.DataFrame({'Target': countries})
df

,Target
0,Istanbul
1,Amsterdam
2,Valletta
3,Basel
4,Doha


In [6]:
df['Distance_km']=df.apply(lambda x: find_d(x['Target'])[0],axis=1)
df['Duration']=df.apply(lambda x: find_d(x['Target'])[1],axis=1)

In [7]:
def get_lat_lng(destination):
    url="https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (destination,api_key)
    try:
        response = requests.get(url)
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        else:
            try:
                response_data = response.json()
            except:
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")
    return (response_data['results'][0]["geometry"]["location"]['lng'],response_data['results'][0]["geometry"]["location"]['lat'])

In [8]:
df['Longitude']=df.apply(lambda x: get_lat_lng(x['Target'])[0],axis=1)
df['Latitude']=df.apply(lambda x: get_lat_lng(x['Target'])[1],axis=1)

In [9]:
df

,Target,Distance_km,Duration,Longitude,Latitude
0,Istanbul,1813,20 hours and 54 minutes,28.978359,41.008238
1,Amsterdam,4536,47 hours and 58 minutes,4.904139,52.367573
2,Valletta,3794,50 hours and 43 minutes,14.514100,35.899237
3,Basel,4095,43 hours and 55 minutes,7.588576,47.559599
4,Doha,2166,22 hours and 44 minutes,51.531040,25.285447


In [10]:
#print(df)

In [11]:
df.sort_values('Distance_km',ascending=False,inplace=True)
print(df['Target'].head(3))

1    Amsterdam
3        Basel
2     Valletta
Name: Target, dtype: object
